In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import pandas as pd
#car = pd.read_csv("/content/gdrive/My Drive/FCA/rulesconfbased.csv",index_col=0)
#car = pd.read_csv("/content/gdrive/My Drive/FCA/Data Files/AllRules_30k/confBasedRules.csv",index_col=0)


car = pd.read_csv("/content/gdrive/My Drive/FCA/Data Files/FinalData1/trans1_confBasedRules.csv")

print(len(car))

39109


In [0]:
print(car.loc[car.index[7], 'premise'])
car.head()

{Correlation, Homogeneity}


,Unnamed: 0,premise,consequence,support,confidence
0,0,{Correlation},{No Finding},0.53,0.53
1,1,{Homogeneity},{No Finding},0.24,0.53
2,2,{Inverse Difference Moment},{No Finding},0.24,0.53
3,3,{Difference Entropy},{No Finding},0.40,0.54
4,4,{Sum Averange},{No Finding},0.18,0.59


In [0]:
def sorter(df):
  len_lhs=[]
  
  for i in df.index:
      p = df.at[i, 'premise']
      if isinstance(p, str):
        p = p.replace('{', '')
        p = p.replace('}', '')
        p = set(p.split(', '))
        df['premise'][i] = p
      
      c = set(df.at[i, "consequence"])
      if isinstance(c, str):
        c = c.replace('{', '')
        c = c.replace('}', '')
        c = set(c.split(', '))
        df['consequence'][i] = c
      
      len_lhs.append(len(list(p)))

  df['LengthLHS'] = len_lhs
  df.sort_values(by=['confidence'], ascending=False, inplace=True)
  return df

In [0]:
car = sorter(car)
car.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


,Unnamed: 0,premise,consequence,support,confidence,LengthLHS
26685,76062,"{Difference Entropy, Cluster Prominence, Entro...",{Effusion},0.0,0.95,6
15595,50815,"{Difference Entropy, Cluster Prominence, Dissi...",{Effusion},0.0,0.95,6
35021,92849,"{Difference Entropy, Cluster Prominence, Dissi...",{Effusion},0.0,0.94,7
26645,75925,"{Sum Entropy, Difference Entropy, Cluster Prom...",{Effusion},0.0,0.94,7
31085,85674,"{Mass, Nodule, Entropy, Consolidation, Atelect...",{Effusion},0.0,0.93,6


In [0]:
#Compute and return jaccard similarity index between premise and instance

def jaccard_sim(premise, instance):
    j = len(premise.intersection(instance))/len(premise.union(instance))
    return j

In [0]:
def check_df(df, feature_set, labels):
  jaccardvalues = {}
  
  for i in df.index:
    premise = df.at[i, 'premise']
    feat_prem = set()

    for el in premise:
      if not(el in labels):
        feat_prem.add(el)

    if not(feat_prem == set()) and feat_prem.issubset(feature_set):
      jac = jaccard_sim(feat_prem, feature_set)
      jaccardvalues.update({i:jac})
  return jaccardvalues

In [0]:
import numpy as np
from sklearn.metrics import roc_auc_score
import operator

def test_jaccard(X, Y, feat_list, labels):  
    
    predictions = []
    ctr = 0
    count = 0
    
    for a in range(len(X)):
      pred = [0]*len(labels)
      feature_set = set()
      for j in range(16):
          #16: n_features
          if X[a][j] == 1:
              feature_set.add(str(feat_list[j]))

      jaccards = check_df(car, feature_set, labels)
      jaccards = dict(sorted(jaccards.items(), key=operator.itemgetter(1),reverse=True))  #sort the dictionary in descending order
      #print(jaccards)

      no_of_values = 51
      top_pairs = {k: jaccards[k] for k in list(jaccards)[:no_of_values]}
      #print(top_pairs)
       
      cons = []
      for key in top_pairs.keys():
        con = car.at[key, 'consequence'] 
        cons.append(con)                #create a list of top 21 consequences 

      counts = dict()
      for i in cons:
        counts[i] = counts.get(i, 0) + 1
      #print(cons)
      #print(counts)

      if bool(counts):
        max_cons = max(counts, key=counts.get) #find the consequence with the maximum count
      #print(max_cons)

        if isinstance(max_cons, str):
          con = con.replace('{','')
          con = con.replace('}','')
          con = set(con.split(', '))
        for c in con:
          pred[labels.index(c)] = 1

      else:
        ctr = ctr + 1
        #print("No match for instance ", a+1)
        #pred[labels.index('No Finding')] = 1

      predictions.append(pred)
      if pred == list(Y[a]):
        count += 1

    print('Correctly classified = ', count)
    predictions = np.array(predictions)

    flag = False
    try:
      auc = roc_auc_score(Y, predictions, average=None)
      flag = True
    except ValueError:
      pass
    if(flag):
      tempdf = pd.DataFrame()
      tempdf['label'] = labels
      tempdf['AUC'] = auc
      print("Class-wise auc : \n", tempdf)

    predictions = predictions.flatten()
    Y = Y.flatten()
    auc_t = roc_auc_score(Y,predictions,average="weighted")
    print("Combined auc : " ,auc_t)
    print("No match found:" , ctr)

In [0]:
# data_frame = pd.read_csv("/content/gdrive/My Drive/FCA/Data Files/FinalData1/test.csv", index_col=0)
data = pd.read_csv("/content/gdrive/My Drive/FCA/Data Files/FinalData1/test.csv")
# data = data_frame.sample(200)
data.head()

,Image Index,Finding Labels,Contrast,Homogeneity,Energy,Correlation,Dissimilarity,ASM,Inverse Difference Moment,Variance,Sum Averange,Sum Entropy,Difference Entropy,Inertia,Cluster Shade,Cluster Prominence,Entropy,Maximum Probability,Mass,No Finding,Nodule,Pneumothorax,Infiltration,Emphysema,Edema,Consolidation,Hernia,Pneumonia,Cardiomegaly,Pleural_Thickening,Fibrosis,Atelectasis,Effusion
0,00000001_002.png,Cardiomegaly|Effusion,13.064677,0.564775,0.042099,0.998599,1.386116,0.001772,0.564775,13.064677,334.268845,9.117170,0.006431,13.064677,-2.344133e+06,1.008983e+09,7.088873,0.011783,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
1,00000002_000.png,No Finding,6.499827,0.464725,0.035710,0.998582,1.673413,0.001275,0.464725,6.499827,360.245327,8.752106,0.007640,6.499827,-8.073136e+05,2.850427e+08,7.205706,0.005526,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,00000003_007.png,Hernia,23.132369,0.355550,0.025374,0.997666,2.598154,0.000644,0.355550,23.132369,265.734624,9.474674,0.010240,23.132369,-7.144502e+05,6.973201e+08,7.954700,0.013064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,00000005_000.png,No Finding,28.039577,0.558369,0.039719,0.996309,1.495486,0.001578,0.558369,28.039577,340.519673,9.149473,0.006894,28.039577,-1.608152e+06,7.056162e+08,7.110231,0.014586,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,00000005_001.png,No Finding,40.673566,0.610719,0.052758,0.991237,1.342741,0.002783,0.610719,40.673566,325.354141,8.347984,0.005798,40.673566,-1.506846e+06,4.677392e+08,6.552632,0.009903,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [0]:
from sklearn.preprocessing import KBinsDiscretizer
import numpy as np

kdis = KBinsDiscretizer(n_bins = 2, encode = "ordinal", strategy = "kmeans")

for c in data.columns[2:18]:
    c_dash = []
    temp = np.reshape(np.array(data[c]), (-1,1))
    c_dash = kdis.fit_transform(temp)
    data[c] = c_dash

In [0]:
X = data.values[:, 2:18].astype(int)
Y = data.values[:, 18:33].astype(int)

In [0]:
feature_names = data.columns[2:18]
label_names = data.columns[18:33]
features = []

#removing space from between the feature names, because thats how they are in rules
for item in feature_names:
    it = item.replace(" ","")
    features.append(it)

test_jaccard(X, Y, list(features), list(label_names))